In [1]:
require 'nn'
require 'UtilsMultiGPU'
json = require 'json'
video = assert(require("libvideo_decoder"))
cmd = torch.CmdLine()
cmd:option('-modelPath', 'video_segmentation.t7', 'Path of model to load')
cmd:option('-videoPath', 'prepare_datasets/dataset/test/7b0b8dea-ac08-11e6-bfe2-d017c2cf90d9.avi', 'Path to the input image to predict on')
cmd:option('-frameNum', 11, 'Number of continuous frames for one sample')
cmd:option('-nGPU', 1)

In [2]:
opt = cmd:parse("")

In [3]:
if opt.nGPU > 0 then
    require 'cunn'
    require 'cudnn'
end

In [4]:
model =  loadDataParallel(opt.modelPath, opt.nGPU)

In [5]:
net_opt = json.load('params.json')

In [6]:
status, height, width, length, fps = video.init(opt.videoPath)

In [7]:
assert(height == net_opt.input_height, 'video height must be equal to the Network input_height')
assert(width == net_opt.input_width, 'video width must be equal to the Network input_width')
assert(opt.frameNum == net_opt.input_channel, 'frameNum must be equal to the Network input_channel')

In [8]:
result = { }
start_frame_idx = 1 + (opt.frameNum - 1) / 2
for x = 1, start_frame_idx - 1 do
    table.insert(result, '-')
end

In [9]:
clip = torch.ByteTensor(opt.frameNum, height, width)
frame_idx = 0
frame = torch.ByteTensor(3, height, width)

In [10]:
while true do
    frame_idx = frame_idx + 1
    print(frame_idx)
    status = video.frame_rgb(frame)
    print("AAAAA")
    if status == false then
        video.exit()
        break
    end
    print("BBBBB")
    if frame_idx < opt.frameNum then
        clip[frame_idx + 1] = image.rgb2y(frame)
        goto continue
    end

    for x = 1, opt.frameNum - 1 do
        clip[x] = clip[x + 1]
    end
    clip[opt.frameNum] = image.rgb2y(frame)

    -- local target_frame_idx = frame_idx - (opt.frameNum - 1) / 2
    predictions = model:forward(clip)
    -- parse the prediction and insert to result table
    print(predictions)
    break
    ::continue::
end

1	


AAAAA	
BBBBB	


2	
AAAAA	
BBBBB	


3	
AAAAA	
BBBBB	
4	


AAAAA	
BBBBB	
5	
AAAAA	
BBBBB	


...

7	
AAAAA	
BBBBB	


8	
AAAAA	
BBBBB	


9	
AAAAA	
BBBBB	
10	


AAAAA	
BBBBB	
11	
AAAAA	
BBBBB	


/home/jesse/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 1 module of nn.Sequential:
In 1 module of nn.Sequential:
...h/install/share/lua/5.1/nn/SpatialDilatedConvolution.lua:31: attempt to index field 'THNN' (a nil value)
stack traceback:
	...h/install/share/lua/5.1/nn/SpatialDilatedConvolution.lua:31: in function <...h/install/share/lua/5.1/nn/SpatialDilatedConvolution.lua:27>
	[C]: in function 'xpcall'
	/home/jesse/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	/home/jesse/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function </home/jesse/torch/install/share/lua/5.1/nn/Sequential.lua:41>
	[C]: in function 'xpcall'
	/home/jesse/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	/home/jesse/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'forward'
	[string "while true do..."]:22: in main chunk
	[C]: in function 'xpcall'
	/home/jesse/torch/install/share/lua/5.1/itorch/main.lua:210: in function </home

In [12]:
clip:size()

  11
 124
 124
[torch.LongStorage of size 3]



In [19]:
m = model:get(1):get(1)

In [21]:
input = torch.Tensor(1, opt.frameNum, height, width):zero()

In [22]:
input[1]:copy(clip)

(1,.,.) = 
 Columns 1 to 15
  171  162  180  190  188  173  187  190  190  189  189  189  192  195  197
  167  179  178  185  186  189  169  193  195  195  196  196  196  195  195
  178  161  174  183  181  182  169  183  197  199  201  202  201  200  197
  178  160  175  175  183  180  181  176  195  196  200  202  204  206  207
  176  178  169  181  192  189  190  186  195  196  197  200  203  207  211
  180  179  164  187  192  188  194  189  199  199  197  197  200  203  207
  175  180  172  175  188  188  192  185  198  198  196  196  198  198  199
  186  177  178  182  186  189  188  187  193  194  196  198  198  196  195
  187  180  179  178  189  188  193  196  192  199  199  203  195  195  199
  186  181  180  181  184  191  195  194  198  196  195  187  198  200  201
  185  184  181  187  177  195  199  191  198  193  199  196  196  198  205
  185  185  185  192  171  198  201  187  196  195  200  201  198  205  198
  185  186  187  194  172  198  200  187  186  203  201  199

   29   29   29   29   29   29   28
  181  183  184  185  186  189  114   27   29   29   29   29   29   29   28
  180  182  185  187  188  191  107   24   30   30   30   30   30   30   29
  181  186  184  186  186  186   81   30   29   29   29   29   29   29   28
  180  185  184  186  186  189   71   31   30   30   30   30   30   30   29
  180  185  184  186  187  193   54   32   30   30   30   30   30   30   29
  179  183  184  186  188  191   39   33   31   31   31   31   31   31   30
  179  183  184  186  189  181   30   34   31   31   31   31   31   31   30
  180  182  185  187  189  164   29   34   32   32   32   32   32   32   31
  181  182  185  187  189  146   32   34   32   32   32   32   32   32   31
  182  182  185  188  188  136   36   33   33   33   33   33   33   33   32
  183  182  183  187  192  111   32   31   32   32   32   32   31   31   31
  183  185  186  188  193   97   32   31   32   32   32   32   31   31   31
  185  187  188  190  194   80   32   31   32   32  

 53   53   51   50   49   49   46   46   45
   53   53   53   53   53   53   53   53   51   50   49   49   46   46   45
   54   54   54   54   54   54   54   54   51   50   49   49   46   46   45
   54   54   54   54   54   54   54   54   51   50   49   49   46   46   45
   56   56   56   56   56   56   56   56   51   50   49   49   46   46   45
   56   56   56   56   56   56   56   56   50   50   49   47   46   45   44
   56   56   56   56   56   56   56   56   50   50   49   47   46   45   44
   56   56   56   56   56   56   56   56   51   50   50   49   47   46   45
   56   56   56   56   56   56   56   56   51   51   50   49   47   46   45
   56   56   56   56   56   56   56   56   52   52   51   50   49   47   46
   56   56   56   56   56   56   56   56   52   52   51   50   49   47   47
   56   56   56   56   56   56   56   56   53   53   52   51   50   49   47
   56   56   56   56   56   56   56   56   53   53   52   51   50   49   47
   59   56   57   54   56   58   53   53   5

  24   24   24   24   24   24   24   24   24   24   24   24   24   24
   22   24   24   24   24   24   24   24   24   24   24   24   24   24   24
   19   24   24   24   24   24   24   24   24   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   25   25   24   24   23   23   22   24   24   24   24   24   24
   24   26   26   26   26   26   26   26   26   25   25   25   25   25   25
   25   26   26   26   26   26   26   26   26   25   25   25   25   25   25
   25   26   26   

5
   41   42   42   42   40   39   39   38   38   39   38   37   37   35   35
   44   43   43   42   42   40   39   39   39   39   38   37   37   35   35
   44   43   43   43   42   42   40   40   39   39   38   37   37   35   35
   45   44   44   43   43   42   40   40   40   39   38   37   37   35   35
   46   44   44   43   43   42   42   40   40   39   38   37   37   35   35
   43   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   43   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   44   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   44   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   45   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   45   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   46   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   46   44   43   42   42   39   39   38   37   37   38   38   38   37   35
   44   46

...

  27   27   27   27   27   27   27
  222  222  222  221  187   13   23   27   27   27   27   27   27   27   27
  222  222  222  221  187   13   23   27   27   27   27   27   27   27   27
  222  222  222  221  187   13   23   27   27   27   27   27   27   27   27
  222  222  222  221  187   13   23   27   27   27   27   27   27   27   27
  222  222  222  221  187   13   23   27   27   27   27   27   27   27   27
  222  222  222  221  187   13   24   29   29   29   29   29   29   29   29
  222  222  222  221  187   13   24   29   29   29   29   29   29   29   29
  223  224  228  225  186   12   24   29   29   29   29   29   29   29   29
  223  224  228  225  186   12   24   29   29   29   29   29   29   29   29
  223  224  228  225  186   12   24   29   29   29   29   29   29   29   29
  223  224  228  225  186   12   24   29   29   29   29   29   29   29   29
  223  224  228  225  186   12   24   29   29   29   29   29   29   29   29
  223  224  228  225  186   12   24   29   29   29   

33   37   37   37   37   37   37   37   36
  223  228  231  233  135   24   33   37   37   37   37   37   37   37   36
  223  231  230  229  118   26   34   37   37   37   37   37   37   37   37
  223  231  230  229  118   26   36   37   37   37   37   37   37   37   37
  223  231  228  227  118   27   36   37   37   37   37   37   37   37   37
  223  230  227  225  117   29   36   37   37   37   37   37   37   37   37
  223  229  224  223  117   30   37   37   37   37   37   37   37   37   37
  223  228  222  221  116   30   37   37   37   37   37   37   37   37   37
  223  228  221  220  116   31   37   37   37   37   37   37   37   37   37
  223  227  220  219  116   31   37   36   36   36   36   36   36   36   36
  222  225  223  223  115   32   37   37   37   37   37   37   37   37   36
  222  225  223  223  115   32   37   37   37   37   37   37   37   37   36
  222  225  223  223  115   32   37   37   37   37   37   37   37   37   36
  222  225  223  223  115   32   37   37   37

   26   26   26   26   26   27   25   25   30   25   31   37  179   65
   27   26   25   26   27   27   29   23   27   30   27   30   33  187   58
   26   25   25   25   27   29   30   26   32   31   27   27   29  187   50
   27   27   27   27   30   31   32   32   34   33   29   27   24  180   48
   30   30   30   30   32   33   34   32   33   33   29   27   24  168   62
   30   30   30   31   33   36   36   29   30   31   30   27   30  150   90
   29   29   30   31   33   34   34   29   31   32   29   29   30  124  122
   29   27   29   30   32   32   32   31   31   30   25   31   24   94  161
   29   29   27   29   31   31   30   31   27   32   24   27   22   79  202
   29   29   27   29   30   30   29   27   24   27   22   26   23   61  203
   28   28   26   28   29   29   28   24   23   24   22   26   27   57  168
   28   28   26   28   29   29   26   24   24   25   24   29   27   55  111
   28   28   26   28   29   30   27   24   26   30   25   29   25   57   57
   28   28   26  

33
   31   31   31   31   31   31   32   32   33   33   33   33   33   33   33
   32   32   32   32   32   32   32   32   33   33   33   33   33   33   33
   32   32   32   32   32   32   33   33   33   33   33   33   33   33   33
   32   32   32   32   32   32   33   33   33   33   33   33   33   33   33
   32   32   32   32   32   32   33   33   33   33   33   33   33   33   33
   32   32   32   32   32   32   33   33   33   33   33   33   33   33   33
   32   32   32   32   32   32   33   33   34   34   34   34   34   33   33
   32   32   32   32   32   32   33   33   37   36   36   36   36   34   34
   32   32   32   32   32   32   34   34   38   37   37   36   36   34   34
   33   33   33   33   33   33   36   36   38   37   37   36   36   34   34
   33   33   33   33   33   33   33   37   37   36   36   34   34   33   32
   34   34   34   34   34   34   34   37   37   36   36   34   34   33   32
   34   34   34   34   34   34   34   37   37   36   36   34   34   33   32
   34   3

   34   34
   36   36   34   34   33   33   32   34   34   34   34   34   34   34   34
   36   36   34   34   33   33   32   34   34   34   34   34   34   34   34
   36   36   34   34   33   33   32   33   33   33   33   32   32   34   34
   36   36   34   34   33   33   32   33   33   32   32   31   31   33   33
   36   36   34   34   33   33   32   33   33   32   32   32   32   32   33
   36   36   34   34   33   33   33   33   32   32   32   32   32   32   32
   34   34   34   33   33   32   34   32   32   32   32   32   32   32   32
   34   33   33   32   32   31   33   32   32   32   32   32   32   32   32
   34   33   33   32   32   31   32   32   32   32   32   32   32   32   32
   34   33   33   32   32   31   32   32   32   32   32   32   32   32   32
   34   33   33   32   32   31   32   32   32   32   32   32   32   32   32
   36   34   34   33   33   32   33   33   32   32   32   32   32   32   32
   36   34   34   33   33   32   33   32   32   32   32   32   32   32   32
 

In [30]:
m1_output = model:get(1):forward(input:cuda())

In [31]:
model:get(2):forward(m1_output)

/home/jesse/torch/install/share/lua/5.1/nn/View.lua:47: input view (1x64x4x4) and desired view (40x-1) do not match
stack traceback:
	[C]: in function 'error'
	/home/jesse/torch/install/share/lua/5.1/nn/View.lua:47: in function 'batchsize'
	/home/jesse/torch/install/share/lua/5.1/nn/View.lua:79: in function 'f'
	[string "local f = function() return model:get(2):forw..."]:1: in main chunk
	[C]: in function 'xpcall'
	/home/jesse/torch/install/share/lua/5.1/itorch/main.lua:210: in function </home/jesse/torch/install/share/lua/5.1/itorch/main.lua:174>
	/home/jesse/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	/home/jesse/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	/home/jesse/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	/home/jesse/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/home/jesse/torch/install/share/lua/5.1/itorch/main.lua:389: in main chunk
	[C]: in function 'require'
	(com